In [1]:
import pandas as pd
import numpy as np

In [2]:
gameInput = pd.read_csv("nhl-game-data/game.csv").dropna()
# goalieInput = pd.read_csv("nhl-game-data/game_goalie_stats.csv")
# skaterStatsInput = pd.read_csv("nhl-game-data/game_skater_stats.csv")
# teamsStatsInput = pd.read_csv("nhl-game-data/game_teams_stats.csv")
# teamInfoInput = pd.read_csv("nhl-game-data/team_info.csv")

In [3]:
gameInput

,game_id,season,type,date_time,date_time_GMT,away_team_id,home_team_id,away_goals,home_goals,outcome,home_rink_side_start,venue,venue_link,venue_time_zone_id,venue_time_zone_offset,venue_time_zone_tz
0,2011030221,20112012,P,2012-04-29,2012-04-29T19:00:00Z,1,4,3,4,home win OT,right,Wells Fargo Center,/api/v1/venues/null,America/New_York,-4,EDT
1,2011030222,20112012,P,2012-05-01,2012-05-01T23:30:00Z,1,4,4,1,away win REG,right,Wells Fargo Center,/api/v1/venues/null,America/New_York,-4,EDT
2,2011030223,20112012,P,2012-05-03,2012-05-03T23:30:00Z,4,1,3,4,home win OT,left,Prudential Center,/api/v1/venues/null,America/New_York,-4,EDT
3,2011030224,20112012,P,2012-05-06,2012-05-06T23:30:00Z,4,1,2,4,home win REG,left,Prudential Center,/api/v1/venues/null,America/New_York,-4,EDT
4,2011030225,20112012,P,2012-05-08,2012-05-08T23:30:00Z,1,4,3,1,away win REG,right,Wells Fargo Center,/api/v1/venues/null,America/New_York,-4,EDT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11429,2018030413,20182019,P,2019-06-02,2019-06-02T00:00:00Z,6,19,7,2,away win REG,left,Enterprise Center,/api/v1/venues/5076,America/Chicago,-5,CDT
11430,2018030414,20182019,P,2019-06-04,2019-06-04T00:00:00Z,6,19,2,4,home win REG,left,Enterprise Center,/api/v1/venues/5076,America/Chicago,-5,CDT
11431,2018030415,20182019,P,2019-06-07,2019-06-07T00:00:00Z,19,6,2,1,away win REG,left,TD Garden,/api/v1/venues/5085,America/New_York,-4,EDT
11432,2018030416,20182019,P,2019-06-10,2019-06-10T00:00:00Z,6,19,5,1,away win REG,left,Enterprise Center,/api/v1/venues/5076,America/Chicago,-5,CDT


In [10]:
# gameAwayValues = gameInput.loc[:,['away_team_id','date_time', 'away_goals','home_goals','season']] 
gameAwayValues = gameInput.loc[:,['away_team_id', 'date_time', 'away_goals', 'home_goals', 'season']].groupby(['away_team_id', 'season']) \
       .agg({'date_time':'size', 'away_goals':'mean', 'home_goals':'mean'}) \
       .rename(columns={'date_time':'count'}) \
       .reset_index()
gameHomeValues = gameInput.loc[:,['home_team_id', 'date_time', 'away_goals', 'home_goals', 'season']].groupby(['home_team_id', 'season']) \
       .agg({'date_time':'size', 'away_goals':'mean', 'home_goals':'mean'}) \
       .rename(columns={'date_time':'count'}) \
       .reset_index()
print(gameAwayValues.head(5))
print(gameHomeValues.head(5))

   away_team_id    season  count  away_goals  home_goals
0             1  20102011     39    1.948718    2.538462
1             1  20112012     53    2.566038    2.528302
2             1  20122013     24    1.916667    2.916667
3             1  20132014     40    2.250000    2.825000
4             1  20142015     41    2.195122    3.073171
   home_team_id    season  count  away_goals  home_goals
0             1  20102011     41    2.463415    2.292683
1             1  20112012     52    2.461538    2.846154
2             1  20122013     24    2.458333    2.750000
3             1  20132014     41    2.243902    2.560976
4             1  20142015     41    2.195122    2.219512


In [4]:
homeTeamStats = pd.DataFrame(columns=['home_team_id', 'season', 'scored', 'conceded'])
awayTeamStats = pd.DataFrame(columns=['away_team_id', 'season', 'scored', 'conceded'])
for index, row in gameAwayValues.iterrows():
    if not gameHomeValues.loc[(gameHomeValues['home_team_id'] == row['away_team_id']) & (gameHomeValues['season'] == row['season'])].empty:
        season = row['season']
        asHomeTeam = gameHomeValues.loc[(gameHomeValues['home_team_id'] == row['away_team_id']) & (gameHomeValues['season'] == row['season'])]
        conceded = (asHomeTeam['away_goals'].values[0]*asHomeTeam['count'].values[0] + row['home_goals']*row['count'])/(asHomeTeam['count'].values[0] + row['count'])
        scored = (asHomeTeam['home_goals'].values[0]*asHomeTeam['count'].values[0] + row['away_goals']*row['count'])/(asHomeTeam['count'].values[0] + row['count'])
        d1 = {'away_team_id': row['away_team_id'], 'season': season, 'conceded': conceded, 'scored': scored}
        d2 = {'home_team_id': asHomeTeam['home_team_id'].values[0], 'season': season, 'conceded': scored, 'scored': conceded}
        homeTeamStats.at[index, :] = d2
        awayTeamStats.at[index, :] = d1
print(homeTeamStats.count())
print(awayTeamStats.count())

home_team_id    265
season          265
scored          265
conceded        265
dtype: int64
away_team_id    265
season          265
scored          265
conceded        265
dtype: int64


In [8]:
gameInput

,game_id,season,type,date_time,date_time_GMT,away_team_id,home_team_id,away_goals,home_goals,outcome,home_rink_side_start,venue,venue_link,venue_time_zone_id,venue_time_zone_offset,venue_time_zone_tz
0,2011030221,20112012,P,2012-04-29,2012-04-29T19:00:00Z,1,4,3,4,home win OT,right,Wells Fargo Center,/api/v1/venues/null,America/New_York,-4,EDT
1,2011030222,20112012,P,2012-05-01,2012-05-01T23:30:00Z,1,4,4,1,away win REG,right,Wells Fargo Center,/api/v1/venues/null,America/New_York,-4,EDT
2,2011030223,20112012,P,2012-05-03,2012-05-03T23:30:00Z,4,1,3,4,home win OT,left,Prudential Center,/api/v1/venues/null,America/New_York,-4,EDT
3,2011030224,20112012,P,2012-05-06,2012-05-06T23:30:00Z,4,1,2,4,home win REG,left,Prudential Center,/api/v1/venues/null,America/New_York,-4,EDT
4,2011030225,20112012,P,2012-05-08,2012-05-08T23:30:00Z,1,4,3,1,away win REG,right,Wells Fargo Center,/api/v1/venues/null,America/New_York,-4,EDT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11429,2018030413,20182019,P,2019-06-02,2019-06-02T00:00:00Z,6,19,7,2,away win REG,left,Enterprise Center,/api/v1/venues/5076,America/Chicago,-5,CDT
11430,2018030414,20182019,P,2019-06-04,2019-06-04T00:00:00Z,6,19,2,4,home win REG,left,Enterprise Center,/api/v1/venues/5076,America/Chicago,-5,CDT
11431,2018030415,20182019,P,2019-06-07,2019-06-07T00:00:00Z,19,6,2,1,away win REG,left,TD Garden,/api/v1/venues/5085,America/New_York,-4,EDT
11432,2018030416,20182019,P,2019-06-10,2019-06-10T00:00:00Z,6,19,5,1,away win REG,left,Enterprise Center,/api/v1/venues/5076,America/Chicago,-5,CDT


In [9]:
dataSet1 = pd.DataFrame(columns=['game_id', 'season', 'date_time', 'game1home','game1away', 'game1isHome','game2home', \
                                 'game2away','game2isHome','game3home','game3away','game3isHome','game4home','game4away',\
                                 'game4isHome','game5home','game5away','game5isHome','home_team_id', 'gameHome', 'gameAway', \
                                 'home_scored_avg', 'home_conceded_avg', 'away_team_id', 'away_scored_avg', 'away_conceded_avg', \
                                 'total'])
# print(gameAwayValues.loc[(gameAwayValues['away_team_id'] == '4') & (gameAwayValues['season'] == '20102011')])
j = 0
for index, row in gameInput.iterrows():
    season = row['season'] - 10001
    if not awayTeamStats.loc[(awayTeamStats['away_team_id'] == row['away_team_id']) & (awayTeamStats['season'] == season)].empty:
        if not homeTeamStats.loc[(homeTeamStats['home_team_id'] == row['home_team_id']) & (homeTeamStats['season'] == season)].empty:
            awayTeam = awayTeamStats.loc[(awayTeamStats['away_team_id'] == row['away_team_id']) & (awayTeamStats['season'] == season)]
            homeTeam = homeTeamStats.loc[(homeTeamStats['home_team_id'] == row['home_team_id']) & (homeTeamStats['season'] == season)]
            d = {'game_id': row['game_id'],\
                 'season': row['season'],\
                 'date_time': row['date_time'],\
                 'home_team_id': row['home_team_id'], \
                 'gameHome': row['home_goals'], \
                 'gameAway': row['away_goals'], \
                 'home_scored_avg': homeTeam['scored'].values[0], \
                 'home_conceded_avg': homeTeam['conceded'].values[0], \
                 'away_team_id': row['away_team_id'], \
                 'away_scored_avg': awayTeam['scored'].values[0], \
                 'away_conceded_avg': awayTeam['conceded'].values[0], \
                 'total': row['home_goals'] + row['away_goals']}
            for i in range(1,6):
                d['game' + str(i) + 'home'] = 0
                d['game' + str(i) + 'away'] = 0
                d['game' + str(i) + 'isHome'] = 1
            if not gameInput.loc[((gameInput['date_time_GMT'] < row['date_time_GMT']) & \
                                  ((gameInput['home_team_id'] == row['home_team_id']) & \
                                  (gameInput['away_team_id'] == row['away_team_id'])) | \
                                  ((gameInput['home_team_id'] == row['away_team_id']) & \
                                  (gameInput['away_team_id'] == row['home_team_id'])))].tail(5).empty:
                previousGames = gameInput.loc[((gameInput['date_time_GMT'] < row['date_time_GMT']) & \
                                  ((gameInput['home_team_id'] == row['home_team_id']) & \
                                  (gameInput['away_team_id'] == row['away_team_id'])) | \
                                  ((gameInput['home_team_id'] == row['away_team_id']) & \
                                  (gameInput['away_team_id'] == row['home_team_id'])))].tail(5)
                i = 1
                for index1, row1 in previousGames.iterrows():
                    if (row1.home_team_id == row.home_team_id):
                        d['game' + str(i) + 'home'] = row1['home_goals']
                        d['game' + str(i) + 'away'] = row1['away_goals']
                        d['game' + str(i) + 'isHome'] = 1
                    else:
                        d['game' + str(i) + 'home'] = row1['away_goals']
                        d['game' + str(i) + 'away'] = row1['home_goals']
                        d['game' + str(i) + 'isHome'] = 0
                    i += 1
                    
            dataSet1.at[index, :] = d
    

In [12]:
dataSet1 = dataSet1.sort_values(by=['date_time'])
dataSet1

,game_id,season,date_time,game1home,game1away,game1isHome,game2home,game2away,game2isHome,game3home,...,game5isHome,home_team_id,gameHome,gameAway,home_scored_avg,home_conceded_avg,away_team_id,away_scored_avg,away_conceded_avg,total
11049,2011020002,20112012,2011-10-06,0,2,0,1,4,0,5,...,0,10,2,0,3.03797,2.60759,8,2.64368,2.55172,2
9192,2011020001,20112012,2011-10-06,1,2,1,3,0,0,2,...,0,6,1,2,2.31731,3.08654,4,3.07692,2.8022,3
11113,2011020004,20112012,2011-10-07,4,3,0,1,5,0,2,...,1,12,1,5,2.96154,2.82051,14,3.04167,2.90625,6
9869,2011020003,20112012,2011-10-07,4,5,0,5,2,0,0,...,0,23,3,4,2.40385,2.95192,5,2.77907,2.48837,7
10104,2011020007,20112012,2011-10-07,0,1,0,3,1,0,4,...,1,7,4,1,2.84884,2.95349,24,2.91765,2.89412,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11429,2018030413,20182019,2019-06-02,2,4,1,2,4,0,3,...,0,19,2,7,2.6875,2.7625,6,3.33333,2.66667,9
11430,2018030414,20182019,2019-06-04,2,4,0,3,2,0,2,...,0,19,4,2,2.6875,2.7625,6,3.33333,2.66667,6
11431,2018030415,20182019,2019-06-07,4,2,1,2,3,1,7,...,0,6,1,2,2.66667,3.33333,19,2.7625,2.6875,3
11432,2018030416,20182019,2019-06-10,3,2,0,2,7,1,4,...,0,19,1,5,2.6875,2.7625,6,3.33333,2.66667,6


In [14]:
dataSetFinal = dataSet1.drop('game_id',1).drop('date_time',1)

In [15]:
dataSetFinal 

,season,game1home,game1away,game1isHome,game2home,game2away,game2isHome,game3home,game3away,game3isHome,...,game5isHome,home_team_id,gameHome,gameAway,home_scored_avg,home_conceded_avg,away_team_id,away_scored_avg,away_conceded_avg,total
11049,20112012,0,2,0,1,4,0,5,4,0,...,0,10,2,0,3.03797,2.60759,8,2.64368,2.55172,2
9192,20112012,1,2,1,3,0,0,2,1,0,...,0,6,1,2,2.31731,3.08654,4,3.07692,2.8022,3
11113,20112012,4,3,0,1,5,0,2,4,1,...,1,12,1,5,2.96154,2.82051,14,3.04167,2.90625,6
9869,20112012,4,5,0,5,2,0,0,4,0,...,0,23,3,4,2.40385,2.95192,5,2.77907,2.48837,7
10104,20112012,0,1,0,3,1,0,4,2,0,...,1,7,4,1,2.84884,2.95349,24,2.91765,2.89412,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11429,20182019,2,4,1,2,4,0,3,2,0,...,0,19,2,7,2.6875,2.7625,6,3.33333,2.66667,9
11430,20182019,2,4,0,3,2,0,2,7,1,...,0,19,4,2,2.6875,2.7625,6,3.33333,2.66667,6
11431,20182019,4,2,1,2,3,1,7,2,0,...,0,6,1,2,2.66667,3.33333,19,2.7625,2.6875,3
11432,20182019,3,2,0,2,7,1,4,2,1,...,0,19,1,5,2.6875,2.7625,6,3.33333,2.66667,6


In [16]:
dataTest = dataSetFinal.loc[dataSetFinal.season == 20182019]

In [17]:
dataTest

,season,game1home,game1away,game1isHome,game2home,game2away,game2isHome,game3home,game3away,game3isHome,...,game5isHome,home_team_id,gameHome,gameAway,home_scored_avg,home_conceded_avg,away_team_id,away_scored_avg,away_conceded_avg,total
7616,20182019,1,4,0,5,4,0,2,0,1,...,0,10,3,2,2.88636,3.34091,8,2.54321,3.18519,5
7617,20182019,5,3,1,1,4,1,5,3,1,...,0,15,7,0,2.80392,3.27451,6,3.33333,2.66667,7
7620,20182019,3,2,1,2,6,0,3,4,1,...,1,7,0,4,3.39506,2.40741,6,3.33333,2.66667,4
7618,20182019,3,4,1,4,3,0,5,1,1,...,1,23,5,2,3.175,2.675,20,2.6875,2.9625,7
7622,20182019,3,2,1,3,4,1,4,2,0,...,0,12,1,2,3.1358,2.77778,2,3.23457,3.64198,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11429,20182019,2,4,1,2,4,0,3,2,0,...,0,19,2,7,2.6875,2.7625,6,3.33333,2.66667,9
11430,20182019,2,4,0,3,2,0,2,7,1,...,0,19,4,2,2.6875,2.7625,6,3.33333,2.66667,6
11431,20182019,4,2,1,2,3,1,7,2,0,...,0,6,1,2,2.66667,3.33333,19,2.7625,2.6875,3
11432,20182019,3,2,0,2,7,1,4,2,1,...,0,19,1,5,2.6875,2.7625,6,3.33333,2.66667,6


In [18]:
dataTrain = dataSetFinal.loc[dataSetFinal.season != 20182019]

In [19]:
dataTrain

,season,game1home,game1away,game1isHome,game2home,game2away,game2isHome,game3home,game3away,game3isHome,...,game5isHome,home_team_id,gameHome,gameAway,home_scored_avg,home_conceded_avg,away_team_id,away_scored_avg,away_conceded_avg,total
11049,20112012,0,2,0,1,4,0,5,4,0,...,0,10,2,0,3.03797,2.60759,8,2.64368,2.55172,2
9192,20112012,1,2,1,3,0,0,2,1,0,...,0,6,1,2,2.31731,3.08654,4,3.07692,2.8022,3
11113,20112012,4,3,0,1,5,0,2,4,1,...,1,12,1,5,2.96154,2.82051,14,3.04167,2.90625,6
9869,20112012,4,5,0,5,2,0,0,4,0,...,0,23,3,4,2.40385,2.95192,5,2.77907,2.48837,7
10104,20112012,0,1,0,3,1,0,4,2,0,...,1,7,4,1,2.84884,2.95349,24,2.91765,2.89412,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,20172018,6,5,1,0,1,1,3,4,0,...,0,15,2,4,2.29474,3.14737,14,2.85366,2.76829,6
634,20172018,6,5,1,0,1,1,3,4,0,...,0,15,2,4,2.29474,3.14737,14,2.85366,2.76829,6
635,20172018,3,0,1,5,6,0,1,0,0,...,0,14,3,2,2.76829,2.85366,15,3.14737,2.29474,5
636,20172018,6,5,1,0,1,1,3,4,0,...,0,15,3,0,2.29474,3.14737,14,2.85366,2.76829,3


In [44]:
dataSet1.to_csv('.\datasets\dataset1.csv', encoding='utf-8', index=False)

In [20]:
dataTest.drop('season',1).to_csv('.\datasets\dataSet1Test.csv', encoding='utf-8', index=False)
dataTrain.drop('season',1).to_csv('.\datasets\dataSet1Train.csv', encoding='utf-8', index=False)